#Belajar Pyspark - Dataframe View dan SQL

Dalam notebook ini kita akan belajar mengenai cara membuat temporary view dan menjalankan SQL query menggunakan view tersebut.

In [ ]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=5f0a129ade07155038f6b132b02f4eabe3596260bcc8dea576108c0b8e5e0ffc
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [20]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


In [ ]:
spark = SparkSession.builder.appName("Belajar PySpark - Temp View & SQL").getOrCreate()

Sebelumnya kita buat DataFrame dari sebuah python list, lalu kita akan lakukan transformasi terhadap DataFrame tersebut.


In [ ]:
data = [['Agus','F',100],['Windy','F',200],
        ['Budi','B',200],['Dina','F',150],
        ['Bayu','F',50],['Dedi','B',50]]

kolom = ["nama","kode_jurusan","nilai"]

df = spark.createDataFrame(data,kolom)

df.show()

+-----+------------+-----+
| nama|kode_jurusan|nilai|
+-----+------------+-----+
| Agus|           F|  100|
|Windy|           F|  200|
| Budi|           B|  200|
| Dina|           F|  150|
| Bayu|           F|   50|
| Dedi|           B|   50|
+-----+------------+-----+



##Membuat Temporary View

In [ ]:
df.createOrReplaceTempView("mahasiswa")

In [21]:
df2 = df.filter(df.kode_jurusan == 'F')
df2.createTempView("mahasiswa")

AnalysisException: ignored

In [ ]:
df2 = df.filter(df.kode_jurusan == 'F')
df2.createOrReplaceTempView("mahasiswa")

##Menjalankan Query SQL

In [ ]:
df.createOrReplaceTempView("mahasiswa")

In [ ]:
result = spark.sql("select * from mahasiswa")
result.show()

+-----+------------+-----+
| nama|kode_jurusan|nilai|
+-----+------------+-----+
| Agus|           F|  100|
|Windy|           F|  200|
| Budi|           B|  200|
| Dina|           F|  150|
| Bayu|           F|   50|
| Dedi|           B|   50|
+-----+------------+-----+



In [ ]:
spark.sql("select * from mahasiswa where kode_jurusan = 'F'").show()

+-----+------------+-----+
| nama|kode_jurusan|nilai|
+-----+------------+-----+
| Agus|           F|  100|
|Windy|           F|  200|
| Dina|           F|  150|
| Bayu|           F|   50|
+-----+------------+-----+



###Menggunakan Fungsi SQL dan Conditional Statement

In [ ]:
spark.sql("""SELECT UPPER(nama) as Nama,
                kode_jurusan as Jurusan,
                nilai as Nilai,
                CASE
                  WHEN nilai > 150 THEN 'High'
                  WHEN nilai > 100 THEN 'Medium'
                  ELSE 'Low'
              END AS Grade
             FROM mahasiswa""").show()

+-----+-------+-----+------+
| Nama|Jurusan|Nilai| Grade|
+-----+-------+-----+------+
| AGUS|      F|  100|   Low|
|WINDY|      F|  200|  High|
| BUDI|      B|  200|  High|
| DINA|      F|  150|Medium|
| BAYU|      F|   50|   Low|
| DEDI|      B|   50|   Low|
+-----+-------+-----+------+



###Agregasi

In [ ]:
spark.sql("""SELECT kode_jurusan,
                    max(nilai) as max_nilai,
                    min(nilai) as min_nilai
             FROM mahasiswa
             GROUP BY kode_jurusan""").show()

+------------+---------+---------+
|kode_jurusan|max_nilai|min_nilai|
+------------+---------+---------+
|           F|      200|       50|
|           B|      200|       50|
+------------+---------+---------+



###Join Reference

In [ ]:
ref = [['F','Fisika','MIPA'],['B','Biologi','MIPA'],
       ['A', 'Akuntansi', 'Ekonomi']]
kolom = ["kode_jurusan","nama_jurusan","nama_fakultas"]
df_ref = spark.createDataFrame(ref,kolom)
df_ref.show()

+------------+------------+-------------+
|kode_jurusan|nama_jurusan|nama_fakultas|
+------------+------------+-------------+
|           F|      Fisika|         MIPA|
|           B|     Biologi|         MIPA|
|           A|   Akuntansi|      Ekonomi|
+------------+------------+-------------+



In [ ]:
df_ref.createOrReplaceTempView("jurusan")

In [ ]:
spark.sql("""SELECT * FROM mahasiswa
          LEFT JOIN jurusan
          ON mahasiswa.kode_jurusan=jurusan.kode_jurusan""").show()

+-----+------------+-----+------------+------------+-------------+
| nama|kode_jurusan|nilai|kode_jurusan|nama_jurusan|nama_fakultas|
+-----+------------+-----+------------+------------+-------------+
| Agus|           F|  100|           F|      Fisika|         MIPA|
|Windy|           F|  200|           F|      Fisika|         MIPA|
| Budi|           B|  200|           B|     Biologi|         MIPA|
| Dina|           F|  150|           F|      Fisika|         MIPA|
| Bayu|           F|   50|           F|      Fisika|         MIPA|
| Dedi|           B|   50|           B|     Biologi|         MIPA|
+-----+------------+-----+------------+------------+-------------+



##Global Temporary View

In [ ]:
df.createOrReplaceGlobalTempView("mhs")

In [ ]:
spark.sql("SELECT * FROM global_temp.mhs").show()

+-----+------------+-----+
| nama|kode_jurusan|nilai|
+-----+------------+-----+
| Agus|           F|  100|
|Windy|           F|  200|
| Budi|           B|  200|
| Dina|           F|  150|
| Bayu|           F|   50|
| Dedi|           B|   50|
+-----+------------+-----+

